### Dynamic Programming without shortage cost 

**Instance 1:**  

    1) Using T_1 = 10
    
    2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.
    
    3) Look under Test Cases and Instance 1, T = 10 for the results
    
**Instance 2:** 
    
    1) Using T_2 = 20
    
    2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.
    
    3) Look under Test Cases and Instance 2, T = 20 for the results

**Instance 3:** 
    
     1) Using T_3 = 30
    
    2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.
    
    3) Look under Test Cases and Instance 1, T = 30 for the results

#### Import Modules

In [1]:
# reading file
import pandas as pd
pd.options.display.width = 1200
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 100

#math and random operations
import random as rd
import math
import numpy as np

# Generating graphs
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

#### Define DP Stages and States and variables

In [2]:
# rd.seed(2844) #Used for random generator seed
T_1,T_2,T_3 = 10, 20, 30 #T values for the final planning horizon period

S = 40 #storage capacity
Q = 40 # max number of units ordered per week
I0 = 0 #starting inventory of 0

# costs for each period
def generateVariables(T):
    ct = [round(rd.uniform(10,20),1) for i in range(T)] # unit ordering costs
    ht = [round(rd.uniform(5,15),1) for i in range(T)] # holding ordering costs
    dt = [int(rd.uniform(0,20)) for i in range(T)] # demand for each period 
    return [ct, ht, dt]

#### Helper functions

In [3]:
def dummy_fill_In(table, tablenum):
    if (tablenum == 1):
        table.append([i for i in range(0,Q+1)])
    else:    
        for row in range(0,S+1):
            list = []
            for col in range(0,Q+1):
                list.append(col)
            table.append(list)
        
def fillIn_StartBoundary(table, tablenum, dt, ht, ct):
    for col in range(0, len(table[tablenum][0])):
        if(col >= dt):
            nextI = I0 + col - dt
            cost = round(ct*col + ht*(I0+col-dt) + OptimalDec(table, 2, nextI, dt),1)
            table[tablenum][0][col] = cost
        else:
            table[tablenum][0][col] = 0
    return table

def fillIn_EndBoundary(table, tablenum, dt, ct):
    for row in range(0, len(table[tablenum])):
        for col in range(0, len(table[tablenum][row])):
            if((col + row) == dt):
                cost = round(ct * col,1)
                table[tablenum][row][col] = cost
            else:
                table[tablenum][row][col] = 0
    return table

def fillIn_Table(table, tablenum, dt, ht, ct):
    for row in range(0, len(table[tablenum])):
        minVal = max(0, dt - row)
        maxVal = min(Q, dt+S-row)
        for col in range(0, len(table[tablenum][row])):
            if(col >= minVal and col < maxVal):
                nextI = row + col - dt
#                 print(f'min={minVal}, max={maxVal}, nextI={nextI}')
                nexttable = tablenum + 1
                cost = round(ct*col + ht*(row+col-dt) + OptimalDec(table, nexttable, nextI, dt),1)
                table[tablenum][row][col] = cost
            elif(col < minVal or col > maxVal):
                table[tablenum][row][col] = 0
    return table

def OptimalDec(table, tablenum, row, dt):
    minVal = max(0, dt - row)
    maxVal = min(Q, dt+S-row)
    opt = 999
    optindex = -1
    for col in range(0,len(table[tablenum][row])):
        if(col >= minVal and col < maxVal):
            if(table[tablenum][row][col] <= opt):
                optindex = col
                opt = table[tablenum][row][col]
        else:
            continue
    return opt

def findMinInRowIndex(T, table, tablenum, row, dt):
    FinalCost = 99999
    indexOfFinalCost = 0
    for col in range(0,len(table[tablenum][row])):
        if((not tablenum == 1) and (not tablenum == T)):
            if(row >= dt):
                indexOfFinalCost = 0
            else:
                if(table[tablenum][row][col] <= FinalCost):
                    FinalCost = table[tablenum][row][col]
                    indexOfFinalCost = col + 1 
        else:
            if(not table[tablenum][row][col] == 0):
                if(table[tablenum][row][col] <= FinalCost):
                    FinalCost = table[tablenum][row][col]
                    indexOfFinalCost = col
    return indexOfFinalCost

def recursive(T, table, tablenum, dt, a, ht, ct):
    if(tablenum == 1):
#         print(f'tablenum={tablenum}, dt={dt[a]}')
        table = fillIn_StartBoundary(table, 1, dt[a], ht, ct)
        return table
    else:
#         print(f'tablenum={tablenum}, dt={dt[a]}')
        if(tablenum == T):
            table = fillIn_EndBoundary(table, T, dt[a], ct)
#             dt = int(rd.uniform(0,20))
            tablenum = tablenum - 1
            a = a - 1
            return recursive(T, table, tablenum, dt, a, ht, ct)
        else:
            table = fillIn_Table(table, tablenum, dt[a], ht, ct)
            tablenum = tablenum - 1
            a = a - 1
            return recursive(T, table, tablenum, dt, a, ht, ct)

#### Solve and Run DP

In [4]:
# %%time
rd.seed(2844)
# dts = []
def createDummyTables(Case_1):
    for i in Case_1:
        dummy_fill_In(Case_1[i], i)
    return Case_1

def RunOnce(table, T, dtlist, a, ht, ct):
    Case_1 = recursive(T, table, T, dtlist, a, ht, ct)
    return Case_1

def findOptimalPath(table, var, Periods):
    n = 0
    li = []
    CurrInv = 0
    NextInv = 0 
    for t in range(1,Periods+1):
        if(t == 1):
            xVal = findMinInRowIndex(Periods, table, t, CurrInv, var[2][n])
            li.append([CurrInv, xVal, var[2][n]])
            NextInv = I0 + xVal - var[2][n]
#             print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, Col={xVal}, n={n}, dt={var[2][n]}')
        else:
            if(n == Periods):
                xVal = findMinInRowIndex(Periods, table, t, CurrInv, var[2][n])
                li.append([CurrInv, xVal, var[2][n]])
                NextInv = (CurrInv + xVal - var[2][n])
#                 print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, Col={xVal}, n={n}, dt={var[2][n]}')
            else:
                n = n + 1
                CurrInv = NextInv
                if(CurrInv >= S):
                    CurrInv = S
                xVal = findMinInRowIndex(Periods, table, t, CurrInv, var[2][n])
                li.append([CurrInv, xVal, var[2][n]])
                NextInv = (CurrInv + xVal - var[2][n])
#                 print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, Col={xVal}, n={n}, dt={var[2][n]}')
    return li

def generateFrame(li, Periods, var, ind):
# Generate Average + Standard Deviation over all the 10 instances, prodcution cost, holding cost, runtime
    totCost = []
    prodCost = []
    holdCost = []
    tables = []
    AverageTotalCost = []
    CurrentInv= []
    UnitsProd = []
    Demand = []
    for i in range(0, len(li)):
        CurrentInv.append(li[i][0])
        UnitsProd.append(li[i][1])
        Demand.append(li[i][2])
    #test instance 1 use 11 and st_1, test instance 2 use 21 and st_2, test instance 3 use 31 and st_3
    for i in range(1, (Periods + 1)): 
        tables.append(i)
        pc = round(var[0][ind]*li[i-1][1],1)
        prodCost.append(pc)
        hc = round(var[1][ind]*(li[i-1][0]+li[i-1][1]-li[i-1][2]),1)
        holdCost.append(hc)
        totCost.append(pc + hc)
    df = pd.DataFrame({
            'Period':tables,
            'Current Inventory':CurrentInv,
            'Units Produced':UnitsProd,
            'Demand':Demand,
            'Production Cost' : prodCost,
            'Holding Cost' : holdCost,
            'Total Cost In Period' : totCost
        })
    return df

In [5]:
# var = generateVariables(T_2)
# # Case_1 ={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]} 
# Case_1 ={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],
#         11:[],12:[],13:[],14:[],15:[],16:[],17:[],18:[],19:[],20:[]} 
# Case_1 = createDummyTables(Case_1)
# table = RunOnce(Case_1, T_2, var[2], 19, var[0][19], var[1][19])
# l = findOptimalPath(table, var, T_2)
# df = generateFrame(l, T_2, var, 19)
# print(l)
# SumCost = df.sum(axis=0)
# AvgProdCost = df.mean(axis=0)
# print(f'Average Production Cost = {round(AvgProdCost[4],1)} \nTotal Cost = {round(SumCost[6],1)}')
# print(df)

### Test Cases

#### Instance 1, T=10

In [6]:
%%time
rd.seed(2844)

var = generateVariables(T_1)
dflist = []
Case_1 ={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]} 
for i in range(9,-1,-1):
    Case_1 = createDummyTables(Case_1)
    table = RunOnce(Case_1, T_1, var[2], 9, var[0][i], var[1][i])
    li = findOptimalPath(table, var, T_1)
    df = generateFrame(li, T_1, var, i)
    dflist.append(df)

AverageTotalCost = 0
AverageProdCost = 0
for i in range(0, T_1):    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print('\n\n',dflist[i])
        SumCost = dflist[i].sum(axis=0)
        AvgProdCost = dflist[i].mean(axis=0)
        AverageTotalCost = AverageTotalCost + SumCost[6]
        AverageProdCost = AverageProdCost + AvgProdCost[4]
        print(f'Average Production Cost = {round(AvgProdCost[4],1)} \nTotal Cost = {round(SumCost[6],1)}')
    
print(f'\n\nAverage Total Cost = {round(AverageTotalCost/10,1)}\nAverage Production Cost = {round(AverageProdCost/10,1)}')



    Period  Current Inventory  Units Produced  Demand  Production Cost  Holding Cost  Total Cost In Period
0       1                  0               4       4             43.2           0.0                  43.2
1       2                  0               4       4             43.2           0.0                  43.2
2       3                  0              16      16            172.8           0.0                 172.8
3       4                  0              18      18            194.4           0.0                 194.4
4       5                  0               0       0              0.0           0.0                   0.0
5       6                  0               7       7             75.6           0.0                  75.6
6       7                  0              14      14            151.2           0.0                 151.2
7       8                  0               7       7             75.6           0.0                  75.6
8       9                  0               

#### Instance 2, T=20

In [7]:
%%time
rd.seed(2844)

var = generateVariables(T_2)
dflist = []
Case_1 ={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],
        11:[],12:[],13:[],14:[],15:[],16:[],17:[],18:[],19:[],20:[]} 
for i in range(19,9,-1):
    Case_1 = createDummyTables(Case_1)
    table = RunOnce(Case_1, T_2, var[2], 19, var[0][i], var[1][i])
    li = findOptimalPath(table, var, T_2)
    df = generateFrame(li, T_2, var, i)
    dflist.append(df)

AverageTotalCost = 0
AverageProdCost = 0
for i in range(0, 10):    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print('\n\n',dflist[i])
        SumCost = dflist[i].sum(axis=0)
        AvgProdCost = dflist[i].mean(axis=0)
        AverageTotalCost = AverageTotalCost + SumCost[6]
        AverageProdCost = AverageProdCost + AvgProdCost[4]
        print(f'Average Production Cost = {round(AvgProdCost[4],1)} \nTotal Cost = {round(SumCost[6],1)}')
    
print(f'\n\nAverage Total Cost = {round(AverageTotalCost/10,1)}\nAverage Production Cost = {round(AverageProdCost/10,1)}')



     Period  Current Inventory  Units Produced  Demand  Production Cost  Holding Cost  Total Cost In Period
0        1                  0              10      10            138.0           0.0                 138.0
1        2                  0              16      16            220.8           0.0                 220.8
2        3                  0               2       2             27.6           0.0                  27.6
3        4                  0              19      19            262.2           0.0                 262.2
4        5                  0              18      18            248.4           0.0                 248.4
5        6                  0               4       4             55.2           0.0                  55.2
6        7                  0               5       5             69.0           0.0                  69.0
7        8                  0              17      17            234.6           0.0                 234.6
8        9                  0     

19      20                  0               8       8            129.6           0.0                 129.6
Average Production Cost = 171.7 
Total Cost = 3434.4


     Period  Current Inventory  Units Produced  Demand  Production Cost  Holding Cost  Total Cost In Period
0        1                  0              10      10            128.0           0.0                 128.0
1        2                  0              16      16            204.8           0.0                 204.8
2        3                  0               2       2             25.6           0.0                  25.6
3        4                  0              19      19            243.2           0.0                 243.2
4        5                  0              18      18            230.4           0.0                 230.4
5        6                  0               4       4             51.2           0.0                  51.2
6        7                  0               5       5             64.0           0.0    

19      20                  0               8       8            108.8           0.0                 108.8
Average Production Cost = 144.2 
Total Cost = 2883.2


     Period  Current Inventory  Units Produced  Demand  Production Cost  Holding Cost  Total Cost In Period
0        1                  0              10      10            195.0           0.0                 195.0
1        2                  0              16      16            312.0           0.0                 312.0
2        3                  0               2       2             39.0           0.0                  39.0
3        4                  0              19      19            370.5           0.0                 370.5
4        5                  0              18      18            351.0           0.0                 351.0
5        6                  0               4       4             78.0           0.0                  78.0
6        7                  0               5       5             97.5           0.0    

#### Instance 3, T=30

In [8]:
%%time
rd.seed(2844)

var = generateVariables(T_3)
dflist = []
Case_1 ={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],
        11:[],12:[],13:[],14:[],15:[],16:[],17:[],18:[],19:[],20:[],
        21:[],22:[],23:[],24:[],25:[],26:[],27:[],28:[],29:[],30:[]} 
for i in range(29,19,-1):
    Case_1 = createDummyTables(Case_1)
    table = RunOnce(Case_1, T_3, var[2], 29, var[0][i], var[1][i])
    li = findOptimalPath(table, var, T_3)
    df = generateFrame(li, T_3, var, i)
    dflist.append(df)

AverageTotalCost = 0
AverageProdCost = 0
for i in range(0, 10):    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print('\n\n',dflist[i])
        SumCost = dflist[i].sum(axis=0)
        AvgProdCost = dflist[i].mean(axis=0)
        AverageTotalCost = AverageTotalCost + SumCost[6]
        AverageProdCost = AverageProdCost + AvgProdCost[4]
        print(f'Average Production Cost = {round(AvgProdCost[4],1)} \nTotal Cost = {round(SumCost[6],1)}')
    
print(f'\n\nAverage Total Cost = {round(AverageTotalCost/10,1)}\nAverage Production Cost = {round(AverageProdCost/10,1)}')



     Period  Current Inventory  Units Produced  Demand  Production Cost  Holding Cost  Total Cost In Period
0        1                  0               6       6             81.0           0.0                  81.0
1        2                  0               5       5             67.5           0.0                  67.5
2        3                  0              17      17            229.5           0.0                 229.5
3        4                  0               9       9            121.5           0.0                 121.5
4        5                  0               7       7             94.5           0.0                  94.5
5        6                  0              16      16            216.0           0.0                 216.0
6        7                  0              15      15            202.5           0.0                 202.5
7        8                  0               9       9            121.5           0.0                 121.5
8        9                  0     

     Period  Current Inventory  Units Produced  Demand  Production Cost  Holding Cost  Total Cost In Period
0        1                  0               6       6             72.6           0.0                  72.6
1        2                  0               5       5             60.5           0.0                  60.5
2        3                  0              17      17            205.7           0.0                 205.7
3        4                  0               9       9            108.9           0.0                 108.9
4        5                  0               7       7             84.7           0.0                  84.7
5        6                  0              16      16            193.6           0.0                 193.6
6        7                  0              15      15            181.5           0.0                 181.5
7        8                  0               9       9            108.9           0.0                 108.9
8        9                  0       